In [314]:
!python -m tdmclient list
!python -m pip install tk-tools
!python -m tdmclient gui

id:       f3394cac-4181-424e-86bd-f1910fa2563b
group id: 9ebd48b4-772b-47c1-92b0-a43394232bba
name:     Thymio 210
status:   2 (available)
cap:      7
firmware: 13



In [315]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

import time
import math

In [316]:
import numpy as np
from numpy import zeros

In [321]:
# parametres
time_rotation = 7
speed_r = 125
speed_f = 125
stop = 0

periode = 0.5

path = [[0, 300], [300, 300], [300,0]]


In [322]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target
    
    motor_left_target = l_speed
    motor_right_target = r_speed

In [270]:
#input : angle to change the trajectory
@tdmclient.notebook.sync_var
def orientation (angle):
    
    global motor_left_target, motor_right_target
    
    if angle < 0:
        motors(speed_r, stop)
    else :
        motors(stop, speed_r)
        
    time.sleep(time_rotation * abs(angle) / (math.pi))
    motors(stop, stop)
    
orientation(np.pi)

In [230]:
def get_angle_between(vec1, vec2):
    vec1_unit = vec1 / np.linalg.norm(vec1)
    vec2_unit = vec2 / np.linalg.norm(vec2)

    return np.arccos(np.dot(vec1_unit, vec2_unit))

In [220]:
#input : coordinates of two different points 
#return the distance between two points 
def distance (x1, y1, x2, y2):
    dist = np.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))
    return dist

In [293]:
def mouving (path):
    teta = zeros(np.shape(path)[0])
    #teta = diff_angle(path)
    #print (teta)

    goalx = path[0][0]
    goaly = path[0][1]
    
    i = 0
    x_pense = 0
    y_pense = 0
    teta_pense = 0
    
    kx = 0
    ky = 0
    kteta = 0
    #[kx, ky, kteta] = kalman()
    
    #vec1 = [0,0]
    #vec2 = [goalx, goaly]
    #angle = get_angle_between(vec2, vec1)
    #orientation(angle)
    #angle = math.atan2(goaly, goalx)
    #print(angle)
    print('x =', x_pense, 'y =', y_pense, 'teta =', teta_pense)

    while i < np.shape(path)[0]-1:
        #[kx, ky, kteta] = kalman()
        [kx, ky, kteta] = [x_pense, y_pense, teta_pense]
        teta[i] = math.atan2((goaly - ky), (goalx - kx)) #change angle according to kalman
        orientation(teta[i] - kteta)
        dist = distance (goalx, goaly, kx, ky)
        if (dist < (speed_f * periode)):
            print('i = ',i)
            i = i + 1
            goalx = path[i][0]
            goaly = path[i][1]
            motors(speed_f, speed_f)
            time.sleep(dist/(speed_f/3))
            motors(stop, stop)
            time.sleep(1)
            
            x_pense = path[i-1][0]
            y_pense = path[i-1][1]
            vec1 = [np.cos(teta[i-1]), np.sin(teta[i-1])]
            vec2 = [goalx - x_pense, goaly - y_pense]
            angle = -get_angle_between(vec1, vec2)
            print('angle', angle)
            teta_pense = angle + teta[i-1]
            orientation(angle)
        else :
            motors(speed_f, speed_f)
            x_pense = kx + speed_f * periode * np.cos(teta[i])
            y_pense = ky + speed_f * periode * np.sin(teta[i])
            teta_pense = teta[i]
            kteta = teta[i]
            print('y =', y_pense)
            print('x=', x_pense)
            print('teta', teta_pense)
            time.sleep(periode)
   
    
    # move the robot to the last point
    while x_pense!=path[i][0] or y_pense!=path[i][1]:
        #[kx, ky, kteta] = kalman()
        [kx, ky, kteta] = [x_pense, y_pense, teta_pense]
        teta[i] = math.atan2((goaly - ky), (goalx - kx))
        print('kx= ',kx,'ky = ', ky,'goalx = ', goalx,'goaly = ',goaly)
        dist = distance(goalx, goaly, kx, ky)
        print('dist = ', dist)
        if (dist < (speed_f * periode)):
            motors(speed_f, speed_f)
            time.sleep(dist/(speed_f/3))
            x_pense=path[i][0]
            y_pense=path[i][1]
        else :
            motors(speed_f, speed_f)
            x_pense = kx + speed_f * periode * np.cos(teta[i])
            y_pense = ky + speed_f * periode * np.sin(teta[i])
            time.sleep(periode)
            
    motors(stop, stop)

In [294]:
mouving(path)

x = 0 y = 0 teta = 0
y = 62.5
x= 3.827021247335479e-15
teta 1.5707963267948966
y = 125.0
x= 7.654042494670958e-15
teta 1.5707963267948966
y = 187.5
x= 1.1481063742006436e-14
teta 1.5707963267948966
y = 250.0
x= 1.4302971815274573e-15
teta 1.5707963267948968
i =  0
angle -1.5707963267948966
y = 300.0
x= 62.5
teta 0.0
y = 300.0
x= 125.0
teta 0.0
y = 300.0
x= 187.5
teta 0.0
y = 300.0
x= 250.0
teta 0.0
i =  1
angle -1.5707963267948966
kx=  300 ky =  300 goalx =  300 goaly =  0
dist =  300.0
kx=  300.0 ky =  237.5 goalx =  300 goaly =  0
dist =  237.5
kx=  300.0 ky =  175.0 goalx =  300 goaly =  0
dist =  175.0
kx=  300.0 ky =  112.5 goalx =  300 goaly =  0
dist =  112.5
kx=  300.0 ky =  50.0 goalx =  300 goaly =  0
dist =  50.0


In [298]:
motors(stop, stop)